<font size="4">  
    ML 5642 Week 3 Assignment: CNN Cancer Detection
</font> 
<font size="3">  
    <ul>
        <li>Dataset is taken from <a href="https://www.kaggle.com/c/histopathologic-cancer-detection/data" target="_blank">Kaggle</a></li>
        <li>The goal of this project is to accurately identify metastatic cancer in small image patches taken from larger digital pathology scans</li>
    </ul>
</font> 

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import Callback

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [ ]:
# connect to Kaggle dataset locally and display contents
#df = pd.read_csv('nytimes_best_restaurants_2024.csv')

train_images = '/kaggle/input/histopathologic-cancer-detection/train/'
test_images = '/kaggle/input/histopathologic-cancer-detection/test/'
label_csv = '/kaggle/input/histopathologic-cancer-detection/train_labels.csv'

df = pd.read_csv(label_csv)

df['id'] = df['id'] + '.tif'
df['label'] = df['label'].astype(str)

print(df.shape)
df.head()